# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
! pip install gmaps

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
output_city_data = pd.read_csv("output_city_data.csv")
output_city_data.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,67.75,71,99,13.22,ZA,1611539396
1,1,hermanus,-34.4187,19.2345,64.99,77,8,8.01,ZA,1611539407
2,2,cockburn town,21.4612,-71.1419,75.97,77,18,17.43,TC,1611539410
3,3,nouadhibou,20.9310,-17.0347,64.40,77,0,16.11,MR,1611539411
4,4,avarua,-21.2078,-159.7750,86.00,62,20,13.80,CK,1611539413


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
gmaps.configure(api_key=g_key)

In [35]:
heat_map_list = output_city_data[["Latitude", "Longitude"]]
heat_map_list


,Latitude,Longitude
0,-33.0153,27.9116
1,-34.4187,19.2345
2,21.4612,-71.1419
3,20.9310,-17.0347
4,-21.2078,-159.7750
...,...,...
551,-14.0833,-70.4333
552,56.1725,107.4458
553,44.3700,38.5200
554,0.5333,72.9333


In [36]:
heat_map_humidity = output_city_data["Humidity"]
heat_map_humidity

0      71
1      77
2      77
3      77
4      62
       ..
551    90
552    87
553    74
554    71
555    52
Name: Humidity, Length: 556, dtype: int64

In [38]:
heat_map = gmaps.figure()
heat_map_layer = gmaps.heatmap_layer(heat_map_list, weights=heat_map_humidity,dissipating=False, max_intensity=250, point_radius=5)
heat_map.add_layer(heat_map_layer)
heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#Ideal weather condition for me are temps greater than 85F and a light breeze of 10mph and 25% cloudiness

ideal_weather_df = output_city_data.loc[(output_city_data["Max Temp"]>85)&(output_city_data["Wind Speed"]<10)&(output_city_data["Cloudiness"]<25)].dropna()
ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,karratha,-20.74,116.85,89.76,25,20,6.38,AU,1605745234
291,291,denpasar,-8.65,115.22,86.00,84,20,2.24,ID,1605745278


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
42,karratha,AU,-20.74,116.85,
291,denpasar,ID,-8.65,115.22,


In [29]:
parameters= {"radius": 5000,
             "types": "lodging",
             "key":g_key
            }
for index, row in hotel_df.iterrows():
    lat=row["Latitude"]
    long=row["Longitude"]
    parameters["location"]=f"{lat},{long}"
    api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_results = requests.get(api_url,params=parameters)
    pprint(hotel_results.json())
    hotel_df.loc[index,"Hotel Name"]=hotel_results.json()["results"][0]["name"]
    
hotel_df

{'html_attributions': [],
 'next_page_token': 'ATtYBwIEncTfKftLvH_23AL8CA5Ds1CqPc5FfZ8fnqa8TyR8_nde7ecfJo82YSPskPuYhDC-ZKSWxTHVtk6iZRRRA1u5Q98H4DEU2P3eHfkFaxfGgYYA4d6xyhFv_jyD2D-SqycCWtfVcxAEQ7CXObz53fr6UA_TlAzDqeRvxQigMtuDaf2FnuXsVPAg6UrH-xOh9lwO0b4uaGIaYbn9tHeuJZ8F6G1rm7HFtBWCsSe2oW42Z1RTSoo7Mi6G5cEM5dI-qYWmpjvvXn5w7VLUpI_9vmf8H6fomxOa819XV3SREmmXJWf8ir06gPp-GYwNNNmqPx1eoZahizuAg41Q-Ed0xpQtuyUYEmCqCwS9zWuoPPJA356jCbr4w8U763ZMR4PgSS8qOKnLys9xFNhG0ELlDkTj4GH2SL56fD9t7BzCPldALCXfPWSO9SYY',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.7322255, 'lng': 116.8467438},
                           'viewport': {'northeast': {'lat': -20.7308124197085,
                                                      'lng': 116.8481896302915},
                                        'southwest': {'lat': -20.7335103802915,
                                                      'lng': 116.8454916697085}}},
              'icon': 'https://maps.gstatic.com/mapfi

{'html_attributions': [],
 'next_page_token': 'ATtYBwI8-HA-TDLfnVK_28juIqSawUJJe8m_i2HMxYp7m_ttIO9bbi0jVSM6oYB-sElIlc8ZeVRvnAQ1u5l50U3Zf8vvkUL7oeYoEA33eNL9r-GE2WqtPcjt3zmPa3pUjRtUL5sN6m2WL1gsaYLQPQKhF5o70ecYqotWO2Oy_qhMwrl8dZCgAXxwKtJeMSUt-tMsejsXd712s_hnvYdNfvfv428wziyYRc8MAP5AF7SS4l_L1uBHKNEHBNrAOcDEMBh_OYj0QwdB2U8-89MyuhUo-e5coTOVocw2oG11lsgCOMpI0NckH4RWEJ8wadCD4f-j7TRAaGo3CjfM5h-lAcVKvmVOLwefB29j1uDrpzSD-VchBlW_oXtlzyCQqPTIuLk2BH9dKihwPZCy7jESfGcVyKBHV1m43CRImyTZGlkwwAPB7_Awci6gzvQo',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -8.639412799999999,
                                        'lng': 115.2061632},
                           'viewport': {'northeast': {'lat': -8.637955719708497,
                                                      'lng': 115.2075144802915},
                                        'southwest': {'lat': -8.6406536802915,
                                                      'lng': 115.2048165197085}}},
      

,City,Country,Latitude,Longitude,Hotel Name
42,karratha,AU,-20.74,116.85,ibis Styles Karratha
291,denpasar,ID,-8.65,115.22,ASTON Denpasar Hotel & Convention Center


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
heat_map.add_layer(marker_layer)
heat_map

Figure(layout=FigureLayout(height='420px'))